# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [2]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [3]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [4]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train_c = X_train.copy()
    X_val_c = X_val.copy()

    if hyperparams['use_pca']:
        # ...
        if hyperparams['pca_standardize']:
            scaler = StandardScaler()
            X_train_c = scaler.fit_transform(X_train_c)
            X_val_c = scaler.transform(X_val_c)
        
        pca = PCA(n_components = hyperparams['pca_components'])
        X_train_pca = pca.fit_transform(X_train_c)
        X_val_pca = pca.transform (X_val_c)

    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train_c = scaler.fit_transform(X_train_c)
        X_val_c = scaler.transform(X_val_c)

    # Aggiunge il termine costante ai dati
    train_ones = np.ones(X_train_c.shape[0])
    val_ones = np.ones(X_val_c.shape[0])

    X_train_ones = np.c_[X_train_c, train_ones]
    X_val_ones = np.c_[X_val_c, val_ones]
    # Calcolo della soluzione di regressione lineare
    teta = (np.linalg.inv(X_train_ones.T @ X_train_ones)) @ X_train_ones.T @ y_train




    if hyperparams['use_regularization']:
        # ...
        lambda_l2= hyperparams['reg_lambda']

        I = np.eye(X_train_ones.shape[1])
        w = (np.linalg.inv(X_train_ones.T @ X_train_ones + lambda_l2 * I)) @ X_train_ones.T @ y_train

    else:
        # ...
        w = np.linalg.inv(X_train_ones.T @ X_train_ones) @ X_train_ones.T @ y_train
        

    # Calcolo predizioni
    # ...
    y_predict_train = X_train_ones @ w
    y_predict_val = X_val_ones @ w

    # Calcola il MAE
    # ...
    y_diff_train = np.abs(y_train - y_predict_train)
    MAE_train = np.mean(y_diff_train)

    y_diff_val = np.abs(y_val -y_predict_val)
    MAE_val = np.mean (y_diff_val)
    return MAE_val, MAE_train, w

# Dividi il dataset in training e test set
# ...
train_fraction = 0.8
train_size = int (train_fraction * X.shape[0])

X_train = X[:train_size]
y_train = y[:train_size]

X_test = X[train_size:]
y_test = y[train_size:]

# Dividi il training set in training set effettivo e validation set
# ...
val_fraction = 0.3
val_size = int(val_fraction * train_size)  
X_val = X_train[train_size - val_size:]
y_val = y_train[train_size - val_size:]

X_train2 = X_train[:train_size - val_size]
y_train2 = y_train[:train_size - val_size]


# Trova la configurazione di iperparametri migliore
# ...
best_mae_val = float('inf')
best_hyperparams = None
best_model = None
mae_train_full = None

for hyperparams in configs:
    mae_val, mae_train, model = pipeline(X_train2, y_train2, X_val, y_val, hyperparams)
    if mae_val < best_mae_val:
        best_mae_val = mae_val
        best_hyperparams = hyperparams
        best_model = model
        mae_train_full = mae_train

# Riallena il modello sul training set completo
# ...

X_train_full = np.concatenate((X_train2, X_val))
y_train_full = np.concatenate((y_train2, y_val))
mae_val, mae_train, best_model = pipeline(X_train_full, y_train_full, X_test, y_test, best_hyperparams)

# Calcola il MAE sul test set
# ...

test_ones = np.ones(X_test.shape[0])
X_test_ones = np.c_[X_test, test_ones]
y_predict_test = X_test_ones @ best_model
y_diff_test = np.abs(y_test - y_predict_test)
MAE_test = np.mean (y_diff_test)

# Stampa  risultati
# ...
print("Migliore configurazione di iperparametri:", best_hyperparams)
print("Miglior MAE sul validation set:", best_mae_val)
print("MAE corrispondente sul training set:", mae_train_full)
print("MAE corrispondente sul test set:", MAE_test)



Migliore configurazione di iperparametri: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 0.1}
Miglior MAE sul validation set: 0.5906915600178654
MAE corrispondente sul training set: 0.6044305436386143
MAE corrispondente sul test set: 0.5484336016242362


/Users/benedettanassi/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(
